# Chiral solutions for Type-II Dirac seesaw

In [1]:
import pandas as pd
import numpy as np
import itertools
import sys
pd.set_option('display.max_colwidth',400)

General functions

In [2]:
from filter_functions import *

## Methodology
1. __Apply__ the conditions and store the list of dictionaries with the solutions in column 'tree_level'. For each dictionary add:
   * List of massless quiral fermions
   * Best extra escalar, `sp` and list of massless fermions after use `sp` → `sp_massless`
2. __Filter__ the solutions with empty `sp_massless` list
3. __Prepre__ equivalent solutions. Creates new columns with the counting of each used particle
5. __Prepare__ output

## Load full solutions

In [3]:
ds=pd.read_json('https://github.com/restrepo/anomaly/raw/main/solutions.json.gz')
ds.shape

(390074, 5)

Check that solutions are unique

In [4]:
ds['strz']=ds['solution'].astype(str)
ds.drop_duplicates('strz').shape

(390074, 6)

In [5]:
ds=ds.drop('strz',axis='columns')

Number of solutions with repeated charges

In [6]:
ds[ds['solution'].apply(lambda l: len(l)-len(set(l)))>0].shape[0]

280959

## 1. Check phenomenological conditions and 2. Filter `sp_massless`
$$ \nu+m+s=0$$

In [7]:
#See anomalysolutions.ipynb for details
#from anomalytools import *

### Prepare functions to filter solutions with at least two sets of repeated charges

In [8]:
def multiple_repeated(ll):
    MR=False
    rp=[]
    triplet=False
    for x in ll:
        if ll.count(x)>1:
            if ll.count(x)>2:
                triplet=True
            rp.append(x)
    rp=sorted(np.unique(rp))
    if len(rp)>=2 and triplet:
        MR=True
    return MR

assert extract_Dirac_and_Majorana([2,6,4,3],8)==[3]

def tree_level_ii(ll):
    '''
    At least a triplet and a doublet is guaranteed
    (m,m,m,...,ν,ν,...)
    '''
    fs=[]
    mrp=[]
    νrp=[]
    for x in ll:
        if ll.count(x)==1:
            fs.append(x)
        elif ll.count(x)==3:
            mrp.append(x)
        elif ll.count(x)>3:
            mrp.append(x)
            fs.append(x)
        elif ll.count(x)==2:
            νrp.append(x)
    mrp=sorted(np.unique(mrp))
    νrp=sorted(np.unique(νrp))
    fs=sorted(np.unique(fs))
    #print(len(mrp),len(νrp),ll)
    sltn=[]
    for m in mrp:
        for ν in νrp:
            #if (mrp[0]+ν in [sum(k)  for k  in itertools.combinations(fs,2)]+list(2*np.array(fs))):
            s=-m-ν# ,fs,[sum(k)  for k  in itertools.combinations(fs,2)],list(2*np.array(fs)))
            nom=mrp.copy()
            kk=nom.remove(m)
            noν=νrp.copy()
            kk=noν.remove(ν)
            massless=fs+nom+noν
            massless=extract_Dirac_and_Majorana(massless,s)
            slt={'[m,ν,s]':[m,ν,s],'massless':massless}
            if len(massless)>0:
                slt['sp'],slt['sp_massless']=best_higgs_singlet(massless)
            else:
                slt['sp'],slt['sp_massless']=(None,None)            
            sltn.append(slt)
    return sltn

#assert best_higgs_singlet([1,-11,9])==(2, [])
#assert tree_level([1, -4, -4, 9, 9,7,7,7,-11])[0]['[l,ν,s,m_in,m_out]']==[-4, 7, -3, 0, -1]
#assert tree_level([ -4, -4,-4, 9, 9,9])[0]['sp']==None

### Initialize filtered solutions

In [9]:
cl=ds.copy()
cl=cl[cl['solution'].apply(multiple_repeated)].sort_values('n').reset_index(drop=True)

Find solutions that satisfy the condition

In [10]:
cl['tree_level']=cl['solution'].apply(tree_level_ii)

Reorder the filtered solutions

In [11]:
cl['smax']=cl['solution'].apply(lambda l:np.abs(l).max())
cl=cl[cl['tree_level'].apply(lambda l: [d for d in l if len(d.get('massless'))==0 or len(d.get('sp_massless'))==0 ] if isinstance(l,list) else None 
               ).apply(len)>0].sort_values(['n','smax']).reset_index(drop=True)

cl['tree_level']=cl['tree_level'].apply(lambda l: [d for d in l if len(d.get('massless'))==0 or 
                                                   len(d.get('sp_massless'))==0 ] if isinstance(l,list) else None 
             )
cl=cl[cl.smax<=30].reset_index(drop=True)
cl[:1]

,l,k,solution,gcd,n,tree_level,smax
0,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,6,"[{'[m,ν,s]': [1, -4, 3], 'massless': [5], 'sp': 10, 'sp_massless': []}]",5


## 3. Flag equivalent solutions

In [12]:
def characterize_solution(cl,i,j=0):
    if j%2==0:
        cl.loc[i,'D_msls']=j
    else:
        cl.loc[i,'M_msls']=j
    #if j==0:
    #    cl.loc[i,'M_msls']=j
        
    cl.loc[i,'rptd']=len( [ cl.loc[i,'solution'].count(x) for x in np.unique(cl.loc[i,'solution']) if cl.loc[i,'solution'].count(x)>1] )
    sl=[d.get('[m,ν,s]') for d in cl.loc[i,'tree_level'] if len(d.get('massless'))==j]
    if len(sl)>0:
        cl.loc[i,'nl']=0
        cl.loc[i,'nν'] =cl.loc[i,'solution'].count(sl[0][1] )
        cl.loc[i,'nr']=0
    return cl

Example: massless solutions

In [13]:
for i in cl[cl['tree_level'].apply(lambda l: [d for d in l if len(d.get('massless'))==0 ] if isinstance(l,list) else None 
             ).apply(len)>0].index:
    cl=characterize_solution(cl,i,0)

### Add solutions with massless fermions that get masses with some $S'$

In [14]:
for j in range(1,6):
    print(j)
    for i in cl[cl['tree_level'].apply(lambda l: [d for d in l if d.get('sp_massless') is not None and len(d.get('sp_massless'))==0  and len(d.get('massless'))==j ] if isinstance(l,list) else None 
             ).apply(len)>0].index:
        cl=characterize_solution(cl,i,j)

1
2
3
4
5


In [15]:
cl['D_msls']=cl['D_msls'].apply(lambda n: 10 if np.isnan(n) else n)
cl['M_msls']=cl['M_msls'].apply(lambda n: 10 if np.isnan(n) else n)
cl['nl']    =cl['nl'    ].apply(lambda n: 10 if np.isnan(n) else n)
cl['nν']    =cl['nν'    ].apply(lambda n: 10 if np.isnan(n) else n)
cl['nr']    =cl['nr'    ].apply(lambda n: 10 if np.isnan(n) else n)
cl['rptd']  =cl['rptd'  ].apply(lambda n: 10 if np.isnan(n) else n)

In [16]:
for k  in ['D_msls','M_msls','nl','nν','nr','rptd']:
    cl[k]=cl[k].astype(int)

In [17]:
cl=cl.sort_values(['n','smax','D_msls','rptd','nl','nν','nr']).reset_index(drop=True)

## 4. Check the rank of the chiral fermion matrix 

In [18]:
def getit(it,dk):
    it['m']=dk.get('[m,ν,s]')[0]
    it['ν']=dk.get('[m,ν,s]')[1]
    it['s']=dk.get('[m,ν,s]')[2]
    it['massless']=dk.get('massless')
    it['sp']=dk.get('sp')
    return it

fl=cl.copy()
tm=pd.DataFrame()
fl=fl.sort_values(['n','smax','D_msls','M_msls']).reset_index(drop=True)
for i in fl.index:
    it=fl.loc[i].to_dict()
    if fl.loc[i,'D_msls']==10:
        k=[d for d in fl.loc[i,'tree_level'] if len(d.get('massless'))==fl.loc[i,'M_msls']]
        if len(k)>0:
            dk=k[0]
            it=getit(it,dk)
            tm=tm.append(it,ignore_index=True)
    elif fl.loc[i,'M_msls']==10:
        k=[d for d in fl.loc[i,'tree_level'] if len(d.get('massless'))==fl.loc[i,'D_msls']]
        if len(k)>0:
            dk=k[0]
            it=getit(it,dk)
            tm=tm.append(it,ignore_index=True)
    else:
        k=[d for d in fl.loc[i,'tree_level'] if len(d.get('massless'))==fl.loc[i,'M_msls']]
        if len(k)>0:
            dk=k[0]
            it=getit(it,dk)
            tm=tm.append(it,ignore_index=True)
        k=[d for d in fl.loc[i,'tree_level'] if len(d.get('massless'))==fl.loc[i,'D_msls']]
        if len(k)>0:
            dk=k[0]
            it=getit(it,dk)
            tm=tm.append(it,ignore_index=True)
        

for k in ['n','gcd','m','ν','s','D_msls','rptd','nl','nν','nr','M_msls']:
    tm[k]=tm[k].astype(int)
tm['nsmax']=tm.apply(lambda row: np.abs( row['solution'] ).max(),axis='columns' )
tm=tm.sort_values(['n','nsmax']).reset_index(drop=True)

#Delete equivalent solutions
#tm['Q']=True
#sltns={'[3, -4, 8, -9]': [3, -4, -6, -6, 7, 7, 8, -9],}
#for k in sltns.keys():
#    iq=tm[((tm['solution'].astype(str)==str(sltns[k])) &  (tm['massless'].astype(str)==k)
#          )].index[0] #unstable dark matter
#    tm.loc[iq,'Q']=False

#tm=tm[tm['Q']].reset_index(drop=True).drop('Q',axis='columns')
tm[['n','l','k','solution','gcd','m','ν','s','massless','sp','M_msls','rptd','nl','nν','nr','D_msls']
  ].sort_values(['n','M_msls','rptd','nl','nν','nr'])#.loc[[0,4]]#.loc[[0,4]]

,n,l,k,solution,gcd,m,ν,s,massless,sp,M_msls,rptd,nl,nν,nr,D_msls
0,6,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,1,-4,3,[5],10.0,1,2,0,2,0,10
1,7,"[-1, 1]","[-1, 0, -1]","[1, 2, 2, -3, -3, -3, 4]",1,-3,2,1,"[1, 4]",5.0,10,2,0,2,0,2
2,7,"[-3, -1]","[-2, -3, 1]","[3, 3, 3, -5, -5, -7, 8]",1,3,-5,2,"[-7, 8]",1.0,10,2,0,2,0,2
3,7,"[9, -6]","[-5, -10, 8]","[9, 9, -24, -24, -24, 25, 29]",45,-24,9,15,"[25, 29]",54.0,10,2,0,2,0,2
4,8,"[1, -3, -2]","[-4, -9, -5, -3]","[1, 3, 3, 3, -5, -7, -7, 9]",1,3,-7,4,[9],18.0,1,2,0,2,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4799,12,"[-2, -1, -2, -1, -2]","[-2, -1, -2, -1, -2]","[5, 5, -8, -8, -16, -16, -16, 19, 19, 22, 22, -28]",2,-16,-8,24,"[-28, 22]",6.0,10,5,0,2,0,2
4800,12,"[-1, -2, -1, -2, 1]","[-1, 2, -2, 2, -2]","[8, 8, -11, -11, -16, -16, 19, 25, 25, 25, -28, -28]",2,25,-16,-9,"[-11, 8]",3.0,10,5,0,2,0,2
4881,12,"[-1, 1, -1, 0, -1]","[-3, 0, -3, 1, -3]","[4, 4, 5, 5, 5, 12, 12, -17, -17, -21, -21, 29]",2,5,-21,16,"[29, -17]",12.0,10,5,0,2,0,2
4882,12,"[-2, 2, -2, 0, -2]","[-3, 0, -3, 2, -3]","[1, 1, 1, -10, -10, 14, 14, -15, -15, 24, 24, -29]",8,1,-15,14,"[-29, 14]",15.0,10,5,0,2,0,2


## Build the final table dropping out equivalent solutions 

In [19]:
fl=pd.DataFrame()

In [20]:
fl=fl.append( tm[ tm['n']==9 ].drop_duplicates(subset=['D_msls','rptd','nl','nν','nr','M_msls']).reset_index(drop=True) )

fl=fl.append( tm[ tm['n']==6 ].drop_duplicates(subset=['D_msls','rptd','nl','nν','nr','M_msls']).reset_index(drop=True) )

fl=fl.append(tm[ tm['n']==7 ].drop_duplicates(subset=['D_msls','rptd','nl','nν','nr','M_msls']).reset_index(drop=True) )

fl=fl.append(tm[ tm['n']==8 ].drop_duplicates(subset=['D_msls','rptd','nl','nν','nr','M_msls']).reset_index(drop=True) )

In [98]:
tm[tm['9']==10]

KeyError: '9'

In [21]:
fl=fl.reset_index(drop=True)

In [22]:
fl.shape

(11, 19)

In [23]:
tmo=tm.copy()

In [24]:
p=np.array( [2,3,5,7,11,13,17,19,23,29,31] )
q=p.copy()
zpq= [ pp*qq for pp in p for qq in q if pp<qq] 
Zpq=sorted( set( zpq ) )

In [25]:
pqs=[ (pp,qq) for pp in p for qq in q if pp<qq]
ZN=dict(zip(Zpq,pqs))

In [26]:
Zpq[:3]

[6, 10, 14]

In [27]:
tmopq=tmo[tmo['s'].apply(lambda s: abs(s) in Zpq)].reset_index(drop=True)

In [28]:
tmopq['Z_N']=tmopq['s'].abs()
tmopq=tmopq.sort_values('Z_N').reset_index(drop=True)

In [29]:
tmopq[['Z_N']].max()

Z_N    39
dtype: int64

In [30]:
dms=tmopq[tmopq['massless'].apply(len)==0].reset_index(drop=True)

In [31]:
row=dms.iloc[0]

In [32]:
def unconditional_stability(row,ZN={6:(2,3),10:(2,5),15:(3,5)},DMs={},check_massless=True):
    '''
    According to appendix of arXiv:1911.05515, 
    generalized to any N
    
    '''
    if check_massless:
        sd=[d for d in row['tree_level'] if d.get('massless')==[]]
    else:
        sd=row['tree_level']
        
    for i in range(len(sd)):
        DMs={}
        d=sd[i]
        N=abs(d['[m,ν,s]'][-1])
        l=[n for n in row['solution'] if n not in d['[m,ν,s]'][:2]]
        #Check for non-hidden chiral fermions
        if 0 in np.array(l)%N:
            DMs={'None':tuple([])}
            continue
        DM=[x for x in itertools.permutations(l,2) 
               if x[0]<=x[1] and abs(sum(x))==N ]
        DM=DM+[(n,n) for n in l if abs(2*n)==N]
        #rank 0
        for dm in DM:
            if dm[0]!=dm[1] and l.count(dm[0])!=l.count(dm[1]) and abs(2*dm[0])!=N and abs(2*dm[0])!=N:
                DMs={'None':tuple([])}
        #missing massles → TODO: Check BUG
        if check_massless and np.setdiff1d(l,[item for sublist in DM for item in sublist]).size>0:
            DMs={'None':tuple([])}
        if not DMs.get('None')==():
            try:
                p,q=ZN[N]
            except KeyError:
                DMs={'None':tuple([])}
                continue
            ps=[n*p for n in range(1,N) if n*p<N]
            qs=[n*q for n in range(1,N) if n*q<N]
            ω=[ sorted([y%N for y in x]) for x in DM]
            DMF=['ψ' if np.setdiff1d(x,ps).size==0 else 'χ' 
                   if np.setdiff1d(x,qs).size==0 else 'None' for x in  ω ]
            DMs=dict(zip(DMF,DM))
            break
    return DMs

In [33]:
#dms.loc[[25]]
dms['DMs']=dms.apply(lambda row: unconditional_stability(row,ZN)  ,axis='columns')

In [34]:
dms[dms['DMs'].apply(lambda d: 'ψ' in d.keys() and 'χ' in d.keys())]

,l,k,solution,gcd,n,tree_level,smax,D_msls,rptd,nl,...,nr,M_msls,m,ν,s,massless,sp,nsmax,Z_N,DMs
0,"[2, -1, 2, 0]","[-3, -2, -3, -2, -3]","[2, -6, -6, -6, -8, -11, 12, 12, 15, 17, -21]",6,11,"[{'[m,ν,s]': [-6, 12, -6], 'massless': [], 'sp': None, 'sp_massless': None}]",21.0,0,2,0,...,0,10,-6,12,-6,[],NaN,21,6,"{'ψ': (-8, 2), 'None': (-11, 17), 'χ': (-21, 15)}"
4,"[-3, 0, -2, 0]","[-2, -4, -2, 1, -1]","[1, 3, 3, 6, 6, 6, -7, -10, -12, -12, 16]",6,11,"[{'[m,ν,s]': [6, -12, 6], 'massless': [], 'sp': None, 'sp_massless': None}, {'[m,ν,s]': [6, 3, -9], 'massless': [-12], 'sp': 24, 'sp_massless': []}]",16.0,0,3,0,...,0,1,6,-12,6,[],NaN,16,6,"{'χ': (3, 3), 'None': (-7, 1), 'ψ': (-10, 16)}"
12,"[-2, 0, -1, 0, 3]","[-2, -1, 0, -1, 1]","[1, -2, 6, 6, 6, -7, 8, -9, -12, -12, 15]",2,11,"[{'[m,ν,s]': [6, -12, 6], 'massless': [], 'sp': None, 'sp_massless': None}]",15.0,0,2,0,...,0,10,6,-12,6,[],NaN,15,6,"{'ψ': (-2, 8), 'None': (-7, 1), 'χ': (-9, 15)}"
16,"[-2, 0, 1, 0, -3]","[-2, -1, 0, -1, 1]","[1, 3, 5, -6, -6, -6, 8, -9, 12, 12, -14]",2,11,"[{'[m,ν,s]': [-6, 12, -6], 'massless': [], 'sp': None, 'sp_massless': None}]",14.0,0,2,0,...,0,10,-6,12,-6,[],NaN,14,6,"{'None': (1, 5), 'χ': (3, 3), 'ψ': (-14, 8)}"


In [35]:
tmopq.shape

(1089, 20)

In [36]:
tmopq.loc[[13]]

,l,k,solution,gcd,n,tree_level,smax,D_msls,rptd,nl,nν,nr,M_msls,m,ν,s,massless,sp,nsmax,Z_N
13,"[-2, -1, 0, -2, -1]","[-2, -1, 2, -2, -1]","[1, 1, -4, -4, -8, -8, 14, 14, 14, 14, -17, -17]",2,12,"[{'[m,ν,s]': [14, -17, 3], 'massless': [14, -8], 'sp': 6, 'sp_massless': []}, {'[m,ν,s]': [14, -8, -6], 'massless': [14, -17, -4, 1], 'sp': 3, 'sp_massless': []}]",17.0,4,5,0,2,0,10,14,-8,-6,"[14, -17, -4, 1]",3.0,17,6


In [37]:
tmopq['DMs']=tmopq.apply(lambda row: unconditional_stability(row,ZN,check_massless=False)  ,axis='columns')

In [64]:
dmss=tmopq[tmopq['DMs'].apply(lambda d: 'ψ' in d.keys() and 'χ' in d.keys())].reset_index(drop=True)

In [65]:
dmss['len_massless']=dmss['massless'].apply(len).reset_index(drop=True)

In [67]:
dmss=dmss.sort_values(['n','len_massless','smax']).reset_index(drop=True)

In [68]:
dmss[dmss['s'].abs()==6]

,l,k,solution,gcd,n,tree_level,smax,D_msls,rptd,nl,...,M_msls,m,ν,s,massless,sp,nsmax,Z_N,DMs,len_massless
0,"[2, 1, 0, 2]","[-3, -5, 3, -3]","[3, 4, -6, -6, -6, -10, 12, 12, 13, -16]",2,10,"[{'[m,ν,s]': [-6, 12, -6], 'massless': [-16, 13], 'sp': 3, 'sp_massless': []}]",16.0,2,2,0,...,10,-6,12,-6,"[-16, 13]",3.0,16,6,"{'ψ': (-10, 4), 'χ': (3, 3)}",2
1,"[-3, -2, 0, 3]","[-4, -2, 2, 0, 1]","[2, -3, 4, 6, 6, 6, -12, -12, -16, 19]",10,10,"[{'[m,ν,s]': [6, -12, 6], 'massless': [-16, 19], 'sp': 3, 'sp_massless': []}]",19.0,2,2,0,...,10,6,-12,6,"[-16, 19]",3.0,19,6,"{'ψ': (2, 4), 'χ': (-3, -3)}",2
2,"[6, -2, 0, 4]","[-1, 1, -6, -4]","[3, 8, 12, 12, 13, -18, -18, -18, -22, 28]",4,10,"[{'[m,ν,s]': [-18, 12, 6], 'massless': [8, 13], 'sp': 21, 'sp_massless': []}]",28.0,2,2,0,...,10,-18,12,6,"[8, 13]",21.0,28,6,"{'ψ': (-22, 28), 'χ': (3, 3)}",2
3,"[-2, 0, 1, 0, -3]","[-2, -1, 0, -1, 1]","[1, 3, 5, -6, -6, -6, 8, -9, 12, 12, -14]",2,11,"[{'[m,ν,s]': [-6, 12, -6], 'massless': [], 'sp': None, 'sp_massless': None}]",14.0,0,2,0,...,10,-6,12,-6,[],NaN,14,6,"{'None': (1, 5), 'χ': (3, 3), 'ψ': (-14, 8)}",0
4,"[-2, 0, -1, 0, 3]","[-2, -1, 0, -1, 1]","[1, -2, 6, 6, 6, -7, 8, -9, -12, -12, 15]",2,11,"[{'[m,ν,s]': [6, -12, 6], 'massless': [], 'sp': None, 'sp_massless': None}]",15.0,0,2,0,...,10,6,-12,6,[],NaN,15,6,"{'ψ': (-2, 8), 'None': (-7, 1), 'χ': (-9, 15)}",0
5,"[-3, 0, -2, 0]","[-2, -4, -2, 1, -1]","[1, 3, 3, 6, 6, 6, -7, -10, -12, -12, 16]",6,11,"[{'[m,ν,s]': [6, -12, 6], 'massless': [], 'sp': None, 'sp_massless': None}, {'[m,ν,s]': [6, 3, -9], 'massless': [-12], 'sp': 24, 'sp_massless': []}]",16.0,0,3,0,...,1,6,-12,6,[],NaN,16,6,"{'χ': (3, 3), 'None': (-7, 1), 'ψ': (-10, 16)}",0
6,"[2, -1, 2, 0]","[-3, -2, -3, -2, -3]","[2, -6, -6, -6, -8, -11, 12, 12, 15, 17, -21]",6,11,"[{'[m,ν,s]': [-6, 12, -6], 'massless': [], 'sp': None, 'sp_massless': None}]",21.0,0,2,0,...,10,-6,12,-6,[],NaN,21,6,"{'ψ': (-8, 2), 'None': (-11, 17), 'χ': (-21, 15)}",0
20,"[-2, 0, -3, 0, -1]","[-2, -1, 1, -1, 0]","[1, -3, 6, 6, 6, -8, -9, 11, -12, -12, 14]",2,11,"[{'[m,ν,s]': [6, -12, 6], 'massless': [-9, 1, 11], 'sp': 2, 'sp_massless': []}]",14.0,10,2,0,...,3,6,-12,6,"[-9, 1, 11]",2.0,14,6,"{'ψ': (-8, 14), 'χ': (-3, -3)}",3
23,"[-2, 1, -3, 0, -3]","[-2, -1, -2, 2, -3]","[3, -8, -9, -10, 14, 14, 14, -15, 16, -20, -20, 21]",12,12,"[{'[m,ν,s]': [14, -20, 6], 'massless': [-8], 'sp': 16, 'sp_massless': []}]",21.0,10,2,0,...,1,14,-20,6,[-8],16.0,21,6,"{'χ': (3, 3), 'ψ': (-10, 16)}",1
33,"[2, -1, 3, -3, 2]","[-3, -2, 0, -1, 0]","[2, -3, -4, -5, 6, 6, 6, -8, 11, -12, -12, 13]",2,12,"[{'[m,ν,s]': [6, -12, 6], 'massless': [-4, 13], 'sp': 9, 'sp_massless': []}]",13.0,2,2,0,...,10,6,-12,6,"[-4, 13]",9.0,13,6,"{'None': (-5, 11), 'ψ': (-8, 2), 'χ': (-3, -3)}",2


In [69]:
dmss.loc[3,'uncstab']=1 # Z_2 Majorana-Dirac
dmss.loc[4,'uncstab']=1 # Z_2 Dirac
dmss.loc[5,'uncstab']=1 # Z_2 Majorana-Majorana
dmss.loc[0,'uncstab']=1 # Majorna

In [70]:
dmss['s'].abs().max()

14

In [71]:
dmss[dmss['s'].abs()==10]

,l,k,solution,gcd,n,tree_level,smax,D_msls,rptd,nl,...,m,ν,s,massless,sp,nsmax,Z_N,DMs,len_massless,uncstab
10,"[1, 3, 1, 0, 1]","[-3, -1, -3, -1, 0]","[2, 2, 2, 3, 4, -5, -5, 6, -12, -12, 15]",3,11,"[{'[m,ν,s]': [2, -12, 10], 'massless': [3], 'sp': 6, 'sp_massless': []}]",15.0,10,3,0,...,2,-12,10,[3],6.0,15,10,"{'ψ': (4, 6), 'χ': (-5, -5)}",1,NaN
11,"[-2, 4, 3, 2]","[-2, -4, 2, 0, 1]","[1, 2, 5, -6, -6, -6, 8, -11, 16, 16, -19]",2,11,"[{'[m,ν,s]': [-6, 16, -10], 'massless': [-19], 'sp': 38, 'sp_massless': []}]",19.0,10,2,0,...,-6,16,-10,[-19],38.0,19,10,"{'ψ': (2, 8), 'None': (-11, 1), 'χ': (5, 5)}",1,NaN
12,"[-3, -1, -2, -3]","[-2, -4, 0, -2, 1]","[2, -3, -3, -3, 5, -11, -12, 13, 13, 21, -22]",2,11,"[{'[m,ν,s]': [-3, 13, -10], 'massless': [-22], 'sp': 44, 'sp_massless': []}]",22.0,10,2,0,...,-3,13,-10,[-22],44.0,22,10,"{'None': (-11, 21), 'ψ': (-12, 2), 'χ': (5, 5)}",1,NaN
15,"[-1, -4, 2, 0]","[-2, -3, -4, -3, 3]","[3, 3, -4, -5, -6, 7, 7, 7, -13, -22, 23]",2,11,"[{'[m,ν,s]': [7, 3, -10], 'massless': [-22], 'sp': 44, 'sp_massless': []}]",23.0,10,2,0,...,7,3,-10,[-22],44.0,23,10,"{'ψ': (-6, -4), 'None': (-13, 23), 'χ': (-5, -5)}",1,NaN
18,"[-2, -3, -1, 0, -2]","[-2, 2, -1, 0, -2]","[2, 4, 4, -5, 6, 6, 6, -12, -16, -16, 21]",2,11,"[{'[m,ν,s]': [6, -16, 10], 'massless': [21, 4], 'sp': 25, 'sp_massless': []}, {'[m,ν,s]': [6, 4, -10], 'massless': [21, -16], 'sp': 5, 'sp_massless': []}]",21.0,2,3,0,...,6,-16,10,"[21, 4]",25.0,21,10,"{'ψ': (-12, 2), 'χ': (-5, -5)}",2,NaN
36,"[3, -3, 2, 1, 3]","[-1, 0, 2, -1, -2]","[1, 1, 1, 2, -4, -5, -6, -7, 9, 9, 15, -16]",1,12,"[{'[m,ν,s]': [1, 9, -10], 'massless': [-16, -7, 2], 'sp': 14, 'sp_massless': []}]",16.0,10,2,0,...,1,9,-10,"[-16, -7, 2]",14.0,16,10,"{'χ': (-5, -5), 'ψ': (-6, -4)}",3,NaN
37,"[-1, -2, -1, 1, -3]","[-3, 0, -2, 0, 2]","[5, 9, 9, 9, 15, -17, -18, -19, -19, 22, -24, 28]",2,12,"[{'[m,ν,s]': [9, -19, 10], 'massless': [-24, -17, 15, 22], 'sp': 2, 'sp_massless': []}]",28.0,4,2,0,...,9,-19,10,"[-24, -17, 15, 22]",2.0,28,10,"{'ψ': (-18, 28), 'χ': (5, 5)}",4,NaN


In [72]:
dmss.loc[10,'uncstab']=1
dmss.loc[36,'uncstab']=1
dmss.loc[37,'uncstab']=1

In [73]:
dmss[dmss['s'].abs()==14]

,l,k,solution,gcd,n,tree_level,smax,D_msls,rptd,nl,...,m,ν,s,massless,sp,nsmax,Z_N,DMs,len_massless,uncstab
7,"[2, -1, 1, 0, 3]","[-3, -1, -2, 3, 0]","[2, 3, -5, -5, 6, 7, 8, -9, -9, -9, 11]",2,11,"[{'[m,ν,s]': [-9, -5, 14], 'massless': [2], 'sp': 4, 'sp_massless': []}]",11.0,10,2,0,...,-9,-5,14,[2],4.0,11,14,"{'None': (3, 11), 'ψ': (6, 8), 'χ': (7, 7)}",1,NaN
8,"[-3, 0, -2, -3, -2]","[-3, -2, 0, 2, -2]","[2, -3, -3, -3, -4, 6, 7, 8, -11, -11, 12]",1,11,"[{'[m,ν,s]': [-3, -11, 14], 'massless': [-4], 'sp': 8, 'sp_massless': []}]",12.0,10,2,0,...,-3,-11,14,[-4],8.0,12,14,"{'ψ': (6, 8), 'χ': (7, 7)}",1,NaN
9,"[-1, 0, -2, 0]","[-2, -4, -2, -1, -2]","[1, -2, -2, -2, 4, -5, 7, 10, -12, -12, 13]",2,11,"[{'[m,ν,s]': [-2, -12, 14], 'massless': [-5], 'sp': 10, 'sp_massless': []}]",13.0,10,2,0,...,-2,-12,14,[-5],10.0,13,14,"{'None': (1, 13), 'ψ': (4, 10), 'χ': (7, 7)}",1,NaN
13,"[1, -3, -1, 0, 1]","[-3, -1, 1, 0, -3]","[2, 2, 2, 5, -7, 8, -16, -16, 21, 21, -22]",3,11,"[{'[m,ν,s]': [2, -16, 14], 'massless': [5], 'sp': 10, 'sp_massless': []}]",22.0,10,3,0,...,2,-16,14,[5],10.0,22,14,"{'χ': (-7, -7), 'ψ': (-22, 8)}",1,NaN
14,"[-2, 0, -4, 0]","[-4, -1, -4, 2, -4]","[2, 2, 4, 7, -8, 9, 10, -16, -16, -16, 22]",40,11,"[{'[m,ν,s]': [-16, 2, 14], 'massless': [9], 'sp': 18, 'sp_massless': []}]",22.0,10,2,0,...,-16,2,14,[9],18.0,22,14,"{'ψ': (-8, 22), 'χ': (7, 7)}",1,NaN
16,"[-1, 0, 2, 0, -2]","[-1, 0, -1, -2, -1]","[2, 5, 5, 5, 5, -7, 9, -10, -19, -19, 24]",1,11,"[{'[m,ν,s]': [5, -19, 14], 'massless': [2], 'sp': 4, 'sp_massless': []}]",24.0,10,2,0,...,5,-19,14,[2],4.0,24,14,"{'ψ': (-10, 24), 'χ': (-7, -7)}",1,NaN
17,"[-1, 0, -2, 0]","[-2, -4, -2, 1, -2]","[3, 6, 6, 6, 7, 11, -12, -13, -20, -20, 26]",2,11,"[{'[m,ν,s]': [6, -20, 14], 'massless': [-13], 'sp': 26, 'sp_massless': []}]",26.0,10,2,0,...,6,-20,14,[-13],26.0,26,14,"{'None': (3, 11), 'ψ': (-12, 26), 'χ': (7, 7)}",1,NaN
19,"[-4, -3, 4, 1]","[-3, -2, -1, -3, -1]","[4, 4, -7, -8, 11, 11, 17, -18, -18, -18, 22]",1,11,"[{'[m,ν,s]': [-18, 4, 14], 'massless': [17, 11], 'sp': 28, 'sp_massless': []}]",22.0,2,3,0,...,-18,4,14,"[17, 11]",28.0,22,14,"{'ψ': (-8, 22), 'χ': (-7, -7)}",2,NaN
21,"[3, 1, -1, 2, 3]","[-2, 1, -2, -1, -2]","[1, 1, 1, -4, 6, -7, -7, 8, -10, 13, 13, -15]",6,12,"[{'[m,ν,s]': [1, -7, 6], 'massless': [-15, -10, -4, 6, 8, 13], 'sp': 2, 'sp_massless': []}, {'[m,ν,s]': [1, 13, -14], 'massless': [-15], 'sp': 30, 'sp_massless': []}]",15.0,10,3,0,...,1,13,-14,[-15],30.0,15,14,"{'ψ': (-10, -4), 'χ': (-7, -7)}",1,NaN
22,"[1, -1, 2, 0, -2]","[-2, 0, -2, -1, 1]","[2, 2, 2, 3, -4, -7, -7, 11, 12, -16, -16, 18]",2,12,"[{'[m,ν,s]': [2, -16, 14], 'massless': [12], 'sp': 24, 'sp_massless': []}]",18.0,10,3,0,...,2,-16,14,[12],24.0,18,14,"{'None': (3, 11), 'ψ': (-4, 18), 'χ': (-7, -7)}",1,NaN


* `.iloc[3]` → Two generations of DM as in Casas talk
* `.iloc[4]` → Only second works 

In [74]:
tm=fl.copy()

In [76]:
tm.shape

(11, 19)

In [77]:
tm=tm[tm.apply(lambda row: rank(row['solution'],row['massless'],row['sp']) if row['sp'] else 0,axis='columns')==0].reset_index(drop=True)

In [78]:
tm.shape

(9, 19)

In [81]:
tm=tm.append(dmss[dmss['uncstab']==1]).reset_index(drop=True) 

In [84]:
tm=tm.sort_values(['n','nsmax']).reset_index(drop=True)

In [85]:
#kk=tm[['n','l','k','solution','gcd','m','ν','s','massless','sp','D_msls','rptd','nl','nν','nr','M_msls']].reset_index(drop=True)
#kk['sltn']=kk['solution'].astype(str)
#kk.sort_values('n').drop('sltn',axis='columns').reset_index(drop=True)

In [88]:
#kk=tm[['n','l','k','solution','gcd','m','ν','s','massless','sp','uncstab']].copy()

In [89]:
import re
def add_boldsymbol(ss):
    if str(ss).find(r'\boldsymbol')==-1:
        return re.sub('(\-*[0-9]+)',r'\\boldsymbol{\1}',str(ss))
    else:
        return ss

In [93]:
kk=tm[['n','l','k','solution','gcd','m','ν','s','massless','sp','uncstab']].copy()

for i in kk.index:
    #for s in ['DD','DM','XD','XM']:
    if not kk.loc[i,'massless']:
        print(i)
        kk.loc[i,'n']=add_boldsymbol(kk.loc[i,'n'])
        #kk.loc[i,s]  =add_boldsymbol(kk.loc[i,s])
        
for i in kk.index:
    #for s in ['DD','DM','XD','XM']:
    if kk.loc[i,'uncstab']==1:
        #print(i)
        kk.loc[i,'s']=add_boldsymbol(kk.loc[i,'s'])         

kkk=kkk.drop('uncstab',axis='columns')#[['n','l','k','solution','gcd']]#Ref','DD','DM','XD','XM']]
def f(x):
    return  r'{}'.format(str(x).replace('[','(').replace(']',')'))

kkk.to_latex('solutions.tex',index=False,formatters=dict( [(k,f) for k in kk.columns ]) ,escape=False  )

7
10
11
13


In [94]:
cat solutions.tex

\begin{tabular}{llllrrrllr}
\toprule
              n &                   l &                   k &                                          solution & gcd &  m &   ν &                s &           massless &   sp \\
\midrule
              6 &            (-1, -2) &             (-1, 2) &                              (1, 1, 1, -4, -4, 5) &   1 &  1 &  -4 &                3 &                (5) & 10.0 \\
              7 &             (-1, 1) &         (-1, 0, -1) &                          (1, 2, 2, -3, -3, -3, 4) &   1 & -3 &   2 &                1 &             (1, 4) &  5.0 \\
              8 &         (1, -3, -2) &    (-4, -9, -5, -3) &                       (1, 3, 3, 3, -5, -7, -7, 9) &   1 &  3 &  -7 &                4 &                (9) & 18.0 \\
              8 &        (-8, -1, -4) &     (-2, -6, 4, -5) &                (7, -8, -18, -18, 20, 20, 20, -23) &   2 & 20 & -18 &               -2 &       (-23, -8, 7) & 16.0 \\
              9 &       (3, 0, -1, 1) &     (-6, -5, 5, -1)

In [95]:
kkk

,n,l,k,solution,gcd,m,ν,s,massless,sp
0,6,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,1,-4,3,[5],10.0
1,7,"[-1, 1]","[-1, 0, -1]","[1, 2, 2, -3, -3, -3, 4]",1,-3,2,1,"[1, 4]",5.0
2,8,"[1, -3, -2]","[-4, -9, -5, -3]","[1, 3, 3, 3, -5, -7, -7, 9]",1,3,-7,4,[9],18.0
3,8,"[-8, -1, -4]","[-2, -6, 4, -5]","[7, -8, -18, -18, 20, 20, 20, -23]",2,20,-18,-2,"[-23, -8, 7]",16.0
4,9,"[3, 0, -1, 1]","[-6, -5, 5, -1]","[2, 2, 2, -3, -3, 4, -5, -5, 6]",1,2,-5,3,[4],8.0
5,9,"[1, -1, -2, -1]","[-6, -5, -3, -5]","[1, 1, 2, 2, 2, -3, -6, -8, 9]",1,2,1,-3,"[-8, -3]",11.0
6,9,"[-2, -3, 0, 1]","[-2, -4, -3, 0]","[1, 4, 5, -6, -6, -6, 9, 9, -10]",1,-6,9,-3,"[-10, 1, 4, 5]",5.0
7,\boldsymbol{9},"[1, 2, 1, -1]","[-6, -3, -5, -3]","[1, 2, -6, -6, -6, 8, 9, 9, -11]",1,-6,9,-3,[],NaN
8,9,"[-5, -2, 3]","[-1, -2, 3, -4]","[3, 5, 5, -8, -8, -8, 12, 12, -13]",3,-8,12,-4,"[-13, 3, 5]",10.0
9,10,"[2, 1, 0, 2]","[-3, -5, 3, -3]","[3, 4, -6, -6, -6, -10, 12, 12, 13, -16]",2,-6,12,\boldsymbol{-6},"[-16, 13]",3.0


In [97]:
assert kkk.shape==(16,10)